# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:


from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
 
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity 
                        WITH REPLICATION = {'class':'SimpleStrategy','replication_factor':1} """)
except Exception as e: 
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e: 
    print(e)

In [10]:
# create table song details . It stores details about song in each session 
# I have decided to use sessionId and itemInSession as primary key since query requires data based on these 2 fields 
# and also it makes eah row unique.
query = "CREATE TABLE IF NOT EXISTS song_details"
query = query + """(sessionId int , itemInSession int ,artist varchar,song varchar, lenght decimal ,
                    PRIMARY KEY(sessionID,itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [11]:
# inserting data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_details(sessionId , itemInSession  ,artist ,song , lenght)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))


#### Display the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [12]:
# Display the result
query = "select artist ,song , lenght from song_details WHERE sessionId=338 and itemInSession=4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist,row.song,row.lenght)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [13]:
# created table artist_library which stores details about song and user who listened the song
# I have decided to use sessionId and userID as primary key since query requires data based on these 2 fields 
# and also it makes eah row unique. I have used itemInSession as clusterning key since we need to display song in 
# order of itemInSession

query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + """(sessionId int, userId int,itemInSession int, artist varchar, 
                    song varchar,firstName varchar,  lastName varchar,
                    PRIMARY KEY((sessionId,userID),itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
# inserting data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library(sessionId , userId ,itemInSession , artist , song ,firstName ,  lastName )"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[10]),int(line[3]),line[0],line[9],line[1],line[4]))



#### Displays the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
# Display the result
query = "select  itemInSession, artist , song ,firstName ,  lastName from artist_library WHERE userid=10 and sessionId = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist,row.song,row.firstname,row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [16]:
# created a table to store user list. 
# I have decided to use song and userID as primary key. Though query requires data based on song however only song 
# is not sufficient to make each row unique. I have added userID to ake each row unique.

query = "CREATE TABLE IF NOT EXISTS user_list "
query = query + """(song varchar, userId int,firstName varchar, lastName varchar,
                    PRIMARY KEY(song,userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
# inserting data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_list(song , userId ,firstName , lastName )"
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### Displays user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [19]:
# Display the result
query = """select firstName , lastName from user_list WHERE song='All Hands Against His Own' """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Dropping the tables

In [4]:
try:
    query = """drop table song_details """
    session.execute(query)
except Exception as e:
    print(e)
    
try:
    query = """drop table artist_library """
    session.execute(query)
except Exception as e:
    print(e)
    
try:
    query = """drop table user_list """
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()